<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/RL/6-4-SpaceInvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium[atari]
!pip install autorom[accept-rom-license]
!AutoROM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 21.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=87c695eb82bb39d6cc901c72b9f9fc97735c2971eaa5dfbb4fd9ab70ddd1a051
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: 

In [ ]:
# import gymnasium as gym
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from collections import deque
# import random
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque
import random
import ale_py # import ale_py to register the ALE namespace

# Hyperparameters
state_size = (210, 160, 3)  # Image size of the Atari games
action_size = 6  # Number of possible actions in reduced space
batch_size = 32
gamma = 0.95  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001

# Create environment
# env = gym.make("ALE/SpaceInvaders-v5")
env = gym.make("ALE/SpaceInvaders-v5", render_mode="rgb_array")

# Define Q-Network model
# def build_model():
#     model = keras.Sequential()
#     model.add(keras.layers.Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=state_size))
#     model.add(keras.layers.Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
#     model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(512, activation='relu'))
#     model.add(keras.layers.Dense(action_size, activation='linear'))
#     model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate), loss='mse')
#     return model
# Define Q-Network model
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=state_size))
    model.add(keras.layers.Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(action_size, activation='linear'))
    # Change 'lr' to 'learning_rate' in the Adam optimizer
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model


In [ ]:
# Training the agent
def train_dqn(episodes):
    model = build_model()
    memory = deque(maxlen=2000)
    # Add this line to declare epsilon as a global variable inside the function
    global epsilon
    for e in range(episodes):
        # state = env.reset()
        # state = np.reshape(state, (1, *state_size))
        state, _ = env.reset() # Get the observation from the tuple returned by env.reset()
        state = np.reshape(state, (1, *state_size)) # Reshape the observation

        for time in range(5000):
            if np.random.rand() <= epsilon:
                action = random.randrange(action_size)
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])

            # Change this line to handle the additional 'truncated' argument
            next_state, reward, terminated, truncated, _ = env.step(action)

            # Set done based on both 'terminated' and 'truncated'
            done = terminated or truncated

            next_state = np.reshape(next_state, (1, *state_size))
            memory.append((state, action, reward, next_state, done))
            state = next_state

            if done:
                print(f"Episode: {e+1}/{episodes}, score: {time}, epsilon: {epsilon:.2}")
                break

            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                for state, action, reward, next_state, done in minibatch:
                    target = reward
                    if not done:
                        target = reward + gamma * np.amax(model.predict(next_state)[0])
                    q_values = model.predict(state)
                    q_values[0][action] = target
                    model.fit(state, q_values, epochs=1, verbose=0)

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

    return model

# Run training
trained_model = train_dqn(100)